In [ ]:
from set_math import *

In [ ]:
from set_visualization import *

# load and merge data

In [ ]:
train = pd.read_csv("../../data/basic/train.csv")
key = pd.read_csv("../../data/basic/key.csv")
weather1 = pd.read_csv("../../data/weather_v8_with_holiday.csv", index_col = 0)

In [ ]:
df = pd.merge(train, key)

In [ ]:
df1 = pd.merge(df, weather1)
df1 = pd.DataFrame(data=df1, columns=['date', 'units','store_nbr','item_nbr', 'station_nbr', 
                                      'tmax', 'tmin', 'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 
                                      'sunrise', 'sunset', 'snowfall', 'preciptotal',
                                      'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed', 
                                      'codesum', 'codesum_YN', 'dayofweek', 'is_holiday'])
print(df1.shape)
print(df1.columns)
df1.head()

# 단 한번도 팔리지 않은 item_nbr 가 있는지 확인.

In [ ]:
df_test = df1

In [ ]:
table = pd.pivot_table(df_test, values = ['units'], index=['store_nbr'], columns=['item_nbr'], aggfunc=np.sum)

In [ ]:
table = table['units']

In [ ]:
sold_item = []
for j in range(1, 45+1):
    for i in range(1, 111+1):
        if table[i].loc[j] != 0:
            sold_item.append(i)

print(sold_item)

In [ ]:
not_sold=[]
for num in range(1, 111+1):
    if num not in sold_item == True:
        not_sold.append(num)
        
print(not_sold)

### 단 한번도 팔리지 않은 아이템은 없다

# units이 0이면 제외

In [ ]:
df_test = df_test[df_test["units"] > 0].reset_index(drop=True)
print(df_test.shape)
df_test.head()

# Trial 0 : units=0을 제외한 모든 데이터 & 모든 독립변수

- 실수형 : 모든 실수형 피쳐
- 카테고리형 : item_nbr, store_nbr

In [ ]:
model_1 = sm.OLS.from_formula('units ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesum_YN + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + C(item_nbr) + C(store_nbr) + codesum_YN + is_holiday + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 1. Correlation & VIF

### 실수형 독립변수 corrleation

In [ ]:
df_test.columns

In [ ]:
dfX1 = df_test[['units', 'tmax', 'tmin',
               'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise','sunset',
               'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
               'resultspeed', 'resultdir', 'avgspeed']]

In [ ]:
fig = plt.figure(figsize=(20, 15))
for i in range(0,18):
    plt.subplot(6, 3, i+1)
    plt.hist(dfX1[dfX1.columns[i]], bins=50)
    plt.xlabel(dfX1.columns[i], fontsize = 20)
plt.tight_layout()

In [ ]:
dfX1_corr = dfX1.corr()

In [ ]:
dfX1_corr

In [ ]:
dfX1_corr['units'] > 0.05

# heat, sunset, resultspeed, avgspeed

In [ ]:
dfX1_corr['units'] < -0.05

#tmin, dewpoint, wetbulb, stnpressure 

##### `units와 그나마 높은 상관관계를 보이는 resultspeed, avgspeed, stnpressure`

### 카테고리 독립변수 one-hot-encoding

In [ ]:
def is_holiday(holiday):
    if holiday == 'holiday':
        return 1
    elif holiday == 'holiday_work':
        return 1
    elif holiday == "workday":
        return 0

In [ ]:
df_test['is_holiday'] = df_test['is_holiday'].apply(is_holiday)

In [ ]:
df_test[['codesum_YN', 'is_holiday']].head()

## 1-1 : Correlation으로 거른 실수형 + 카테고리형 독립변수

In [ ]:
df_test.columns

In [ ]:
features = ['date', 'store_nbr', 'item_nbr', 'units', 'station_nbr', 'tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'codesum', 'codesum_YN', 'snowfall', 'preciptotal',
       'stnpressure', 'sealevel', 'resultspeed', 'resultdir', 'avgspeed',
       'dayofweek', 'is_holiday']
features = " ".join(features)
features = features.replace(" ", " + ")
features

### 1-1-1

- 실수형 : resultspeed, avgspeed, stnpressure
- 카테고리 : item_nbr, store_nbr

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultspeed + avgspeed + stnpressure  + C(item_nbr) + C(store_nbr)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 1-1-2

- 실수형 : resultspeed, avgspeed, stnpressure
- 카테고리 : item_nbr, store_nbr, codesum_YN

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultspeed + avgspeed + stnpressure + codesum_YN + C(item_nbr) + C(store_nbr)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 1-1-3

- 실수형 : resultspeed, avgspeed, stnpressure
- 카테고리 : item_nbr, store_nbr, is_holiday

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultspeed + avgspeed + stnpressure + is_holiday + C(item_nbr) + C(store_nbr)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 1-1-4

- 실수형 : resultspeed, avgspeed, stnpressure
- 카테고리 : item_nbr, store_nbr, is_holiday, codesum_YN

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultspeed + avgspeed + stnpressure + is_holiday+ codesum_YN + C(item_nbr) + C(store_nbr)', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 1-2 : VIF 로 거른 실수형 + 카테고리형 독립변수


#### VIF를 통한 실수형 독립변수 정리

In [ ]:
df_for_vif = df_test[['tmax', 'tmin',
       'tavg', 'depart', 'dewpoint', 'wetbulb', 'heat', 'cool', 'sunrise',
       'sunset', 'snowfall', 'preciptotal', 'stnpressure', 'sealevel',
       'resultspeed', 'resultdir', 'avgspeed']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns

In [ ]:
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['tavg']

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['sealevel']

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['wetbulb']

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['sunset']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['stnpressure']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['tmin']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['tmax']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['sunrise']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

In [ ]:
del df_for_vif['avgspeed']
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df_for_vif.values, i) for i in range(df_for_vif.shape[1])]
vif["features"] = df_for_vif.columns
vif.sort_values(by=['VIF Factor'], ascending=False).reset_index(drop=True)

dewpoint, resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart

depart, heat, cool, sunrise, snowfall, preciptotal,  resultspeed,  resultdir

### 1-2-1

- 실수형 : dewpoint, resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart
- 카테고리 : item_nbr, store_nbr

In [ ]:
model_1 = sm.OLS.from_formula('units ~ dewpoint + resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 1-2-2

- 실수형 : resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart
- 카테고리 : item_nbr, store_nbr, is_holiday

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 2-3

- 실수형 : resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart
- 카테고리 : item_nbr, store_nbr, codesum_YN

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + codesum_YN + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

### 2-4

- 실수형 : resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart
- 카테고리 : item_nbr, store_nbr, codesum_YN, is_holiday

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + codesum_YN + is_holiday + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# 최종 결정

### 1-2-2

- 실수형 : resultdir, resultspeed, cool, heat, preciptotal, snowfall, depart
- 카테고리 : item_nbr, store_nbr, is_holiday

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0', data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

# Scaling 적용

In [ ]:
model_s = sm.OLS.from_formula('units ~ scale(resultdir) + scale(resultspeed) + scale(cool) + scale(heat) + scale(preciptotal) + scale(snowfall) + scale(depart) + C(item_nbr) + C(store_nbr) + is_holiday + 0', data=df_test)
result_s = model_s.fit()
print(result_s.summary())

In [ ]:
model_s = sm.OLS.from_formula('units ~ scale(resultspeed) + scale(cool) + scale(heat) + scale(preciptotal) + scale(depart) + C(item_nbr) + C(store_nbr) + is_holiday + 0', data=df_test)
result_s = model_s.fit()
print(result_s.summary())

# Outlier 제거

In [ ]:
# 1차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0',data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

In [ ]:
# 2차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0',data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

In [ ]:
# 3차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0',data=df_test)
result_1 = model_1.fit()
print(result_1.summary())

In [ ]:
# 4차
res = result_1.resid_pearson
index = result_1.resid.index
outlier_index = abs(res) >= 2

row = pd.Series(index=index, data=res)
drop_index = row[outlier_index].index

In [ ]:
df_test.drop(list(drop_index),inplace=True)

In [ ]:
model_1 = sm.OLS.from_formula('units ~ resultdir + resultspeed + cool + heat + preciptotal + snowfall + depart + C(item_nbr) + C(store_nbr) + is_holiday + 0',data=df_test)
result_1 = model_1.fit()
print(result_1.summary())